In [1]:
import rasterio as rio
import geopandas as gpd
import pandas as pd
import os, sys, glob
from osgeo import gdal
from rasterstats import zonal_stats
from rasterio.warp import reproject, Resampling
from shapely.geometry import box
from rasterio.mask import mask
import numpy as np
from rasterio import features
from shapely.geometry import shape, Polygon
from shapely.wkt import loads
from tqdm import tqdm
import geojson, json
import osmnx as ox
from affine import Affine
from matplotlib import pyplot as plt
from rasterio.plot import show
from shapely.wkt import loads
from shapely.geometry import Point

In [2]:
sys.path.append("/home/wb514197/Repos/INFRA_SAP")
import infrasap.rasterMisc as rMisc
import infrasap.aggregator as agg

In [11]:
data_dir = '/home/wb514197/data/ethiopia'
dig_dir = os.path.join(data_dir, "digitize_africa")

In [4]:
woreda = os.path.join(data_dir, 'admin', 'gadm', 'ETH_adm3.shp')
flood_map = os.path.join(data_dir, "flood", "Ethiopia_S1S2_fusion_20m_cog.tif")

In [5]:
target = gpd.read_file(woreda)
flood = rio.open(flood_map)

In [15]:
buildings = [gpd.read_file(f) for f in glob.glob(dig_dir+"/*.shp")]

In [16]:
len(buildings)

6

In [19]:
flood.crs

CRS.from_epsg(32636)

In [20]:
buildings = [b.to_crs('EPSG:32636') for b in buildings]

In [23]:
buildings_all = pd.concat(buildings)

In [27]:
sindex_buildings = buildings_all.sindex

In [30]:
flood_crs = flood.crs.to_string()

In [31]:
flood_data = flood.read(1)

In [32]:
flood_mask = (flood_data==1)

In [33]:
allFeatures = []
idx = 0
for cShape, value in features.shapes(flood_data, mask=flood_mask, transform=flood.meta['transform']):
    allFeatures.append([idx, shape(geojson.loads(json.dumps(cShape)))])
    idx+=1

In [34]:
xx = pd.DataFrame(allFeatures, columns=['ID', 'geometry'])
xxGeom = gpd.GeoDataFrame(xx, geometry='geometry', crs = flood_crs)

In [35]:
sindex_flood = xxGeom.sindex

In [38]:
target = target.to_crs(flood_crs)

In [39]:
adm0 = target.unary_union

In [41]:
geom_cut = ox.utils_geo._quadrat_cut_geometry(adm0, 50000)

In [48]:
res = []

In [47]:
xxGeom.loc[:,"bool"] = 1

In [49]:
for i, poly in tqdm(enumerate(geom_cut)):
    
    possible_matches_buildings_index = list(sindex_buildings.intersection(poly.bounds))
    possible_matches_flood_index = list(sindex_flood.intersection(poly.bounds))

    possible_matches_buildings = buildings_all.iloc[possible_matches_buildings_index]
    possible_matches_flood = xxGeom.iloc[possible_matches_flood_index]

    buildings_filt = possible_matches_buildings[possible_matches_buildings.intersects(poly)]
    flood_filt = possible_matches_flood[possible_matches_flood.intersects(poly)].copy()
    
    intersect = gpd.sjoin(buildings_filt, flood_filt, how='left', rsuffix="flood")
    intersect = intersect[intersect['bool']==1].copy()
    res.append(intersect)

540it [3:03:58, 20.44s/it]  


In [50]:
results = pd.concat(res)

In [51]:
len(results)

39038

In [54]:
target.columns

Index(['ID_0', 'ISO', 'NAME_0', 'ID_1', 'NAME_1', 'ID_2', 'NAME_2', 'ID_3',
       'NAME_3', 'TYPE_3', 'ENGTYPE_3', 'NL_NAME_3', 'VARNAME_3', 'geometry'],
      dtype='object')

In [55]:
results2 = gpd.sjoin(results, target[['NAME_3', 'ID_3', 'geometry']], how='left', rsuffix="admin")

In [57]:
results2.columns

Index(['PolyDate', 'PolySource', 'ImgSource', 'ImgDate', 'MosaicName',
       'geometry', 'index_flood', 'ID', 'bool', 'index_admin', 'NAME_3',
       'ID_3'],
      dtype='object')

In [63]:
results2.head()

,PolyDate,PolySource,ImgSource,ImgDate,MosaicName,geometry,index_flood,ID,bool,index_admin,NAME_3,ID_3
4580604,2020-06,Ecopia,Maxar,2016-01,CUSTOM_ETHIOPIA_BBA_20Q1,"POLYGON ((1172755.662 397652.512, 1172760.119 ...",543310.0,543310.0,1.0,242,Moyale,243
4587752,2020-06,Ecopia,Maxar,2016-01,CUSTOM_ETHIOPIA_BBA_20Q1,"POLYGON ((1172717.653 397656.680, 1172721.727 ...",543311.0,543311.0,1.0,242,Moyale,243
4587567,2020-06,Ecopia,Maxar,2016-01,CUSTOM_ETHIOPIA_BBA_20Q1,"POLYGON ((1172764.314 397662.122, 1172767.402 ...",543310.0,543310.0,1.0,242,Moyale,243
4587566,2020-06,Ecopia,Maxar,2016-01,CUSTOM_ETHIOPIA_BBA_20Q1,"POLYGON ((1172765.874 397666.976, 1172766.763 ...",543310.0,543310.0,1.0,242,Moyale,243
4587560,2020-06,Ecopia,Maxar,2016-01,CUSTOM_ETHIOPIA_BBA_20Q1,"POLYGON ((1172765.835 397666.952, 1172762.728 ...",543310.0,543310.0,1.0,242,Moyale,243


In [68]:
results3 = results2.loc[~results2.index.duplicated(keep='first')].copy()

In [69]:
results3.fillna(0, inplace=True)

In [72]:
summary_count = results3.groupby('ID_3')[['ImgSource']].count()

In [82]:
out_folder = os.path.join(data_dir, "output")
if not os.path.exists(out_folder):
    os.mkdir(out_folder)

In [73]:
summary_count

,ImgSource
ID_3,
1,44
2,37
3,136
4,84
5,2
...,...
561,3
563,56
564,6


In [74]:
results3.loc[:, "Area"] = results3.geometry.area

In [75]:
summary_area = results3.groupby('ID_3')[['Area']].sum()

In [76]:
summary_area

,Area
ID_3,
1,6903.776632
2,6499.153251
3,15713.880267
4,7996.021927
5,2202.693591
...,...
561,48.367311
563,3375.843408
564,518.376233


In [79]:
summary = summary_count.join(summary_area, how='outer')

In [83]:
summary.to_csv(os.path.join(out_folder, "buildings_flooded.csv"))

In [141]:
buildings_all2 = gpd.sjoin(buildings_all, target[['NAME_3', 'ID_3', 'geometry']], how='left', rsuffix="admin")

In [153]:
buildings_all2.columns

Index(['PolyDate', 'PolySource', 'ImgSource', 'ImgDate', 'MosaicName',
       'geometry', 'index_admin', 'NAME_3', 'ID_3'],
      dtype='object')

In [144]:
all_buildings_count = buildings_all2.groupby('ID_3')[['ImgSource']].count()

In [150]:
summary2 = summary.join(all_buildings_count, how='outer', rsuffix='_all')

In [152]:
summary2.to_csv(os.path.join(out_folder, "buildings_flooded.csv"))

In [154]:
summary2

,ImgSource,Area,ImgSource_all
ID_3,,,
1.0,44.0,6903.776632,5724
2.0,37.0,6499.153251,4877
3.0,136.0,15713.880267,6701
4.0,84.0,7996.021927,7796
5.0,2.0,2202.693591,9415
...,...,...,...
563.0,56.0,3375.843408,83777
564.0,6.0,518.376233,70513
565.0,34.0,2733.989145,50256


In [98]:
roads_esri = gpd.read_file(os.path.join(out_folder, "roads_flooded.shp"))

In [99]:
roads_esri.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [100]:
roads_esri = roads_esri.to_crs(flood.crs)

In [101]:
roads_esri['type'].unique()

array(['unclassified', 'primary', 'tertiary', 'secondary', 'residential',
       'trunk', 'track', 'primary_link', 'service', 'path', 'trunk_link',
       'footway', 'construction', 'motorway', 'tertiary_link',
       'pedestrian', 'steps', 'secondary_link'], dtype=object)

In [102]:
allowed_types = ['unclassified', 'primary', 'tertiary', 'secondary', 'residential',
       'trunk', 'track', 'primary_link', 'service', 'trunk_link',
       'footway', 'motorway', 'tertiary_link',
       'pedestrian', 'steps', 'secondary_link']

In [103]:
roads_esri = roads_esri.loc[roads_esri['type'].isin(allowed_types)].copy()

In [104]:
roads_esri.loc[:, "length"] = roads_esri.geometry.length

In [109]:
roads_esri2 = gpd.sjoin(roads_esri, target[['NAME_3', 'ID_3', 'geometry']], how='left', rsuffix="admin")

In [112]:
tab_flood = roads_esri2.groupby('ID_3')[['length']].sum()

In [126]:
# roads_all = gpd.read_file('/home/wb514197/data/INFRA_SAP/ETH/highways.shp')
roads_all = gpd.read_file(os.path.join(out_folder, "roads_all.shp"))

In [127]:
roads_all['type'].unique()

array(['residential', 'primary', 'service', 'tertiary', 'trunk',
       'secondary', 'unclassified', 'primary_link', 'trunk_link', 'path',
       'track', 'secondary_link', 'steps', 'footway', 'tertiary_link',
       'platform', 'motorway_link', 'construction', 'motorway',
       'rest_area', 'pedestrian', 'bridleway', 'road', 'raceway',
       'living_street', 'bus_guideway'], dtype=object)

In [128]:
allowed_types = ['residential', 'primary', 'service', 'tertiary', 'trunk',
       'secondary', 'unclassified', 'primary_link', 'trunk_link',
       'track', 'secondary_link', 'footway', 'tertiary_link',
       'platform', 'motorway_link', 'motorway', 'pedestrian', 
        'bridleway', 'road', 'raceway',
       'living_street', 'bus_guideway']

In [129]:
roads_all = roads_all.loc[roads_all['type'].isin(allowed_types)].copy()

In [130]:
roads_all = roads_all.to_crs(flood.crs)

In [131]:
roads_all.loc[:, "length"] = roads_all.geometry.length

In [132]:
tab_all = roads_all.groupby('ID_3')[['length']].sum()

In [134]:
tab_all

,length
ID_3,
1,29554.878212
2,36501.484861
3,47216.909758
4,47624.950176
5,43916.889572
...,...
563,716970.127568
564,456880.385127
565,231529.383718


In [136]:
roads_tab = tab_all.join(tab_flood, rsuffix="_flood", how='outer')

In [137]:
roads_tab.fillna(0, inplace=True)

In [138]:
roads_tab.to_csv(os.path.join(out_folder, "roads_affected.csv"))